Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [886]:
import numpy as np
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

In [887]:
class Play:
    state:[]
    position:int
    
    def __init__(self, position, state):
        self.position = position
        self.state = state
        pass
    

class Player:
    def play(self,turn, token, state)->Play:
        pass 
    
    def wintrain(self, token_winner, token):
        pass
    def win(self):
        pass

class Game:
    play_list :[Play] =[0 for i in range(0,9)]
    player_list :[Player] = []
    
    def __init__(self, playerr1, playerr2):
        self.player_list.append(playerr1)
        self.player_list.append(playerr2)
        pass
    
    def starttrain(self):
        
        turn = 0
        token = 1
        currentplayer = self.player_list[0]
        self.play_list[turn] = currentplayer.play(turn, token,[0 for i in range(0,9)])
        turn = 1
        token *= -1
        currentplayer = self.player_list[1]
            
        while turn < 3 or not(self.win(self.play_list[turn-1].state,token*-1)) and turn < 9 :
            self.play_list[turn] = currentplayer.play((turn-(turn%2==1))//2,token, self.play_list[turn-1].state)
            turn +=1
            token*=-1
            currentplayer = self.player_list[0] if token == 1 else self.player_list[1]
            
        if self.win(self.play_list[turn-1].state,token*-1):
            self.player_list[0].wintrain(token*-1,1)
            self.player_list[1].wintrain(token*-1,-1)
        else:
            self.player_list[0].wintrain(0,1)
            self.player_list[1].wintrain(0,-1)
        pass
    
    def win(self,elements,token):
        """Checks is elements is winning"""
        listdumb = []
        for i in range(0,len(elements)):
            if elements[i] == token:
                listdumb.append(i+1)
        return any(sum(c) == 15 for c in combinations(listdumb, 3))

    def trygame(self):
        turn = 0
        token = 1
        currentplayer = self.player_list[0]
        self.play_list[turn] = currentplayer.play(turn, token,[0 for i in range(0,9)])
        k = [self.play_list[turn].state[1] ,
                 self.play_list[turn].state[6] ,
                 self.play_list[turn].state[5] ,
                 self.play_list[turn].state[8] ,
                 self.play_list[turn].state[4] ,
                 self.play_list[turn].state[0] ,
                 self.play_list[turn].state[3] ,
                 self.play_list[turn].state[2] ,
                 self.play_list[turn].state[7] 
            ]
        for r in range(3):
            for c in range(3):
                y = r*3 +c
                if k[y] == 1:
                    print('X', end='')
                elif k[y] == -1:
                    print('O', end='')
                else:
                    print('.', end='')
            print()
        print()
        turn = 1
        token *= -1
        currentplayer = self.player_list[1]
            
        while turn < 3 or not(self.win(self.play_list[turn-1].state,token*-1)) and turn < 9 :
            self.play_list[turn] = currentplayer.play((turn-(turn%2==1))//2,token, self.play_list[turn-1].state)
            k = [self.play_list[turn].state[1] ,
                 self.play_list[turn].state[6] ,
                 self.play_list[turn].state[5] ,
                 self.play_list[turn].state[8] ,
                 self.play_list[turn].state[4] ,
                 self.play_list[turn].state[0] ,
                 self.play_list[turn].state[3] ,
                 self.play_list[turn].state[2] ,
                 self.play_list[turn].state[7] 
            ]
            for r in range(3):
                for c in range(3):
                    y = r*3 +c
                    if k[y] == 1:
                        print('X', end='')
                    elif k[y] == -1:
                         print('O', end='')
                    else:
                        print('.', end='')
                print()
            print()
            turn +=1
            token*=-1
            currentplayer = self.player_list[0] if token == 1 else self.player_list[1]
            
        if self.win(self.play_list[turn-1].state,token*-1):
            self.player_list[0].win()
            self.player_list[1].win()
            print('player who has token',token*-1,'win (like if it is 1 this is good)')
            return token*-1
        else:
            self.player_list[0].win()
            self.player_list[1].win()
            print('nobody win')
            return 0
    

In [888]:
class RandomPlayer(Player):
    def play(self, turn ,token, state) ->Play:
        dumblist = []
        for i in range(0,len(state)):
            if state[i]==0:
                dumblist.append(i)
        choosen = choice(dumblist)
        state[choosen] = 1*token
        return Play(choosen, state)
        
    def wintrain(self, token_winner, token):
        pass
    def win(self):
        pass

In [889]:
class Cell:
    position:int
    rewardCount = 0
    stateWin = []
    nbStateWin = 0
    
    def __init__(self, position, stateWin = None):
        self.position=position
        if stateWin is not None:
            self.stateWin=stateWin
        pass
        
    
class RenforcementPlayer(Player):
    memoryCell:[[Cell]]
    renforcementCell:[Cell] = []
    
    def __init__(self):
        self.memoryCell = [[Cell(j) for j in range(1,10)]for i in range(0,5)]
    
    def play(self, turn, token, state) ->Play:
        state = [i*token for i in state]
        memorylist = sorted(self.memoryCell[turn], key=lambda x:x.rewardCount,reverse=True)
        for i in range(0,9):
            if memorylist[i].stateWin == state:
                if abs(memorylist[i].rewardCount-memorylist[0].rewardCount)<=20 :
                    memorylist = [memorylist[i]] + memorylist[:i]+memorylist[i+1:]
                    for j in range(0,9):
                        if state[memorylist[j].position-1] == 0 :
                            state[memorylist[j].position-1] = 1
                            self.renforcementCell.append(Cell(memorylist[j].position,state))    
                            state = [i*token for i in state]
                            return Play(memorylist[j].position,state)
                else:
                     for j in range(0,9):
                        if state[memorylist[j].position-1] == 0 :
                            state[memorylist[j].position-1] = 1
                            self.renforcementCell.append(Cell(memorylist[j].position,state))    
                            state = [i*token for i in state]
                            return Play(memorylist[j].position,state)
            
                
        
        
        if abs(memorylist[1].rewardCount-memorylist[0].rewardCount)<=10:
            randlist=[]
            for i in range(0,9):
                if state[memorylist[i].position-1] == 0:
                    randlist.append(i)
            truc = choice(randlist)
            state[truc] = 1
            self.renforcementCell.append(Cell(truc+1,state))    
            state = [i*token for i in state]
            return Play(truc+1,state)
        
        for i in range(0,9):
            if state[memorylist[i].position-1] == 0 :
                state[memorylist[i].position-1] = 1
                self.renforcementCell.append(Cell(memorylist[i].position,state))    
                state = [i*token for i in state]
                return Play(memorylist[i].position,state)
        
    def win(self):
        self.renforcementCell=[]
        pass
        
    
    def wintrain(self, token_winner, token):
        count = 1 if token == -1 else 0
        if token_winner == token:
            for i in self.renforcementCell:
                self.memoryCell[count][i.position-1].rewardCount += 2 + count
                if self.memoryCell[count][i.position-1].stateWin == i.stateWin :
                    self.memoryCell[count][i.position-1].nbStateWin +=2
                else:
                    if self.memoryCell[count][i.position-1].nbStateWin <= 0: 
                        self.memoryCell[count][i.position-1].stateWin = i.stateWin
                        self.memoryCell[count][i.position-1].nbStateWin = 0
                    else:
                        self.memoryCell[count][i.position-1].nbStateWin -= 2
                count+=1
        elif token == 0:
            for i in self.renforcementCell:
                self.memoryCell[count][i.position-1].rewardCount +=1
                count+=1
        else:
            for i in self.renforcementCell:
                self.memoryCell[count][i.position-1].rewardCount -= 2 + count
            
                if self.memoryCell[count][i.position-1].stateWin == i.stateWin :
                    self.memoryCell[count][i.position-1].nbStateWin -=2
                count+=1
            
        self.renforcementCell = []
    pass
                    
                

In [890]:
player1 = RandomPlayer()
player2 = RenforcementPlayer()
gametictac = Game(player2,player1)
for z in range(0,2000):
    gametictac.starttrain()
print('train done')


train done


In [891]:
for l in player2.memoryCell:
    for m in l :
        print(m.rewardCount, " ", end='')
    print()


-12  1006  -32  18  18  20  -68  -26  -24  
-33  60  -78  1425  234  -60  -87  -48  -63  
-100  -88  -88  -100  2076  -84  84  184  -84  
-165  -20  -160  -5  -75  1455  -180  160  1020  
-270  -102  -90  -102  -126  12  -132  -36  -174  


In [896]:
bong = [0,0,0]
for u in range(0,150):
    resu = gametictac.trygame()
    if resu == -1:
        bong[0] +=1
    elif resu == 0:
        bong[1] +=1
    else:
        bong[2] +=1
print("result is : loose, equality, win n :",bong)

X..
...
...

XO.
...
...

XO.
...
X..

XO.
..O
X..

XO.
.XO
X..

XO.
.XO
X.O

XOX
.XO
X.O

player who has token 1 win (like if it is 1 this is good)
X..
...
...

X..
..O
...

X..
..O
X..

X..
.OO
X..

X..
.OO
X.X

X..
OOO
X.X

player who has token -1 win (like if it is 1 this is good)
X..
...
...

X..
...
.O.

X..
...
XO.

X.O
...
XO.

X.O
.X.
XO.

X.O
OX.
XO.

X.O
OX.
XOX

player who has token 1 win (like if it is 1 this is good)
X..
...
...

X..
..O
...

X..
..O
X..

X..
.OO
X..

X..
.OO
X.X

X..
OOO
X.X

player who has token -1 win (like if it is 1 this is good)
X..
...
...

X.O
...
...

X.O
...
X..

X.O
..O
X..

X.O
.XO
X..

X.O
.XO
XO.

X.O
XXO
XO.

player who has token 1 win (like if it is 1 this is good)
X..
...
...

X.O
...
...

X.O
...
X..

X.O
.O.
X..

X.O
.O.
X.X

XOO
.O.
X.X

XOO
XO.
X.X

player who has token 1 win (like if it is 1 this is good)
X..
...
...

X..
...
.O.

X..
...
XO.

X.O
...
XO.

X.O
.X.
XO.

X.O
OX.
XO.

X.O
OX.
XOX

player who has token 1 win (like if it 